# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
%pip install pymongo
from pymongo import MongoClient
cursor = MongoClient()
cursor
str_conn = 'mongodb://localhost:27017'
cursor = MongoClient(str_conn)

Note: you may need to restart the kernel to use updated packages.


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [25]:
# Your Code

db = cursor.companies
collection = db.collection

query = {'name': 'Babelgum'}
select = {'name': 1, '_id': 0}
print(list(collection.find(query, select)))

[{'name': 'Babelgum'}]


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [37]:
# Your Code

query = {'number_of_employees': {'$gt':5000}}
select = {'number_of_employees': 1, '_id': 0}
list(collection.find(query, select).limit(20))

[{'number_of_employees': 5299},
 {'number_of_employees': 15000},
 {'number_of_employees': 63000},
 {'number_of_employees': 13600},
 {'number_of_employees': 28000},
 {'number_of_employees': 86300},
 {'number_of_employees': 5080},
 {'number_of_employees': 7173},
 {'number_of_employees': 180500},
 {'number_of_employees': 300000},
 {'number_of_employees': 25000},
 {'number_of_employees': 8000},
 {'number_of_employees': 8657},
 {'number_of_employees': 33350},
 {'number_of_employees': 125000},
 {'number_of_employees': 10000},
 {'number_of_employees': 90000},
 {'number_of_employees': 80000},
 {'number_of_employees': 8000},
 {'number_of_employees': 51000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [4]:
# Your Code

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [5]:
# Your Code

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [6]:
# Your Code

### 6. All the companies that don't include the `partners` field.

In [7]:
# Your Code

### 7. All the companies that have a null type of value on the `category_code` field.

In [8]:
# Your Code

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [9]:
# Your Code

### 9. Order all the companies by their IPO price in a descending order.

In [10]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [11]:
# Your Code

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [12]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [13]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [14]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [15]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [16]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [17]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [18]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [19]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [20]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [21]:
# Your Code